In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/churn-predictions-personal/Churn_Predictions.csv


In [4]:
import numpy as np
import pandas as pd
import tensorflow as tf

2024-04-24 17:33:44.100038: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-24 17:33:44.100243: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-24 17:33:44.349724: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [5]:
tf.__version__

'2.15.0'

In [6]:
dataset= pd.read_csv('/kaggle/input/churn-predictions-personal/Churn_Predictions.csv')

In [7]:
X= dataset.drop("Exited",axis=1).values
y=dataset["Exited"].values

#.values converts the above datasets into numPy arrays

In [8]:
# But we don't need the first two columns as they are irrelevant
#(contain names of customers and customer id)
X=dataset.iloc[:,3:-1].values #from index 3 till 1 before last
y=dataset.iloc[:,-1].values #last column

In [9]:
print(X)

[[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]]


In [10]:
print(y)

[1 0 1 ... 1 1 0]


### Encoding Categorical Data
##### Label Encoding the "gender" column

In [11]:
from sklearn.preprocessing import LabelEncoder
le= LabelEncoder()
X[:,2]=le.fit_transform(X[:,2]) #transforming the first 2 columns/categorical data

In [12]:
print(X)
# As we can see below female was encoded into 0 and male was encoded into 1

[[619 'France' 0 ... 1 1 101348.88]
 [608 'Spain' 0 ... 0 1 112542.58]
 [502 'France' 0 ... 1 0 113931.57]
 ...
 [709 'France' 0 ... 0 1 42085.58]
 [772 'Germany' 1 ... 1 0 92888.52]
 [792 'France' 0 ... 1 0 38190.78]]


In [13]:
# One Hot Encoding of the 'Geography' column.
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
X = np.array(ct.fit_transform(X))

In [14]:
print(X)
# We can see across the rows: France encoded as 1.0 0.0 0.0,
# Spain as 0.0 0.0 1.0 and Germany as 0.0 1.0 0.0

[[1.0 0.0 0.0 ... 1 1 101348.88]
 [0.0 0.0 1.0 ... 0 1 112542.58]
 [1.0 0.0 0.0 ... 1 0 113931.57]
 ...
 [1.0 0.0 0.0 ... 0 1 42085.58]
 [0.0 1.0 0.0 ... 1 0 92888.52]
 [1.0 0.0 0.0 ... 1 0 38190.78]]


In [15]:
#Splitting into Training and Test Set
from sklearn.model_selection import train_test_split
X_train, X_test,y_train,y_test= train_test_split(X,y,test_size=0.2,random_state=0)

In [16]:
#Feature Scaling
from sklearn.preprocessing import StandardScaler
sc= StandardScaler()
X_train= sc.fit_transform(X_train)
X_test=sc.transform(X_test)

#### Part 2- Building the ANN
* Initializing the ANN
* Adding the Input layer and the first hidden layer
* Adding the second hidden layer
* Adding the output layer

In [17]:
# Initializing the ANN
ann= tf.keras.models.Sequential()
#This creates and ann variable which represents our artificial neural network
#represented as a sequence of layers.

### Tips for below code:
##### Activation function in a fully connected neural network must be rectifier activation function.
##### units=nodes= hyperparameters for the first hidden layer below, which we have to find out by experimentation, no rule of thumb for it.

In [18]:
#Addding the input layer and the first hidden layer
#This adds a fully connected layer into an artificial neural network
#at whatever state/phase of the artificial neural network where we are.
ann.add(tf.keras.layers.Dense(units=6,
                             activation='relu'))
#This is a new object/instance of the dense class.
# So the fully connected layer we are about to build will be created as an object of the dense class.

In [19]:
#Adding the second hidden layer:

ann.add(tf.keras.layers.Dense(units=6,
                             activation='relu'))

In [20]:
# In the output layer, for binary classification we need just 1 unit or 1 neuron,
# but for more than 2 (non binary )dependent variables like A,B,C we will use three
#different codes like 001 100 010 and therefore we will need 3 neurons.

# For output layer we need sigmoid activation, which gives us probabilities of the outcome.

In [21]:
# Output layer:
#Explanation above:
ann.add(tf.keras.layers.Dense(units=1,activation='sigmoid'))

### Part 3- Training the ANN
* Compiling the ANN
* Training the ANN on the Training set

##### From the ANN object( object/instance) of the sequential class, we are going to call the `compile` method.

#### Explanation for below code:
* `Adam` optimizer is one of the best optimizers working on Stochastic Gradient
* For binary classification we use `binary_crossentropy`,
* For non binary crossentropy we use `crossentropy`,
* For predicting more than 2 categories, the activation should be `softmax`.


In [22]:
ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [23]:
#fit method of the sequential class' ann object always takes the same parameters.
#x matrix for training and y for the dependent variable prediction.

In [24]:
ann.fit(X_train, y_train, batch_size = 32, epochs = 100)

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7489 - loss: 0.5934
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7944 - loss: 0.4818
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8005 - loss: 0.4474
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7957 - loss: 0.4487
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7879 - loss: 0.4453
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7968 - loss: 0.4385
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7968 - loss: 0.4248
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7944 - loss: 0.4316
Epoch 9/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8123 - loss: 0.4198
Epoch 10/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8081 - loss: 0.4392
Epoch 11/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8198 - loss: 0.4108
Epoch 12/100
250/250 ━━━━━━━━━━━━━━━━━━━━

#### Predict customer churn for the below data:

Use our ANN model to predict if the customer with the following informations will leave the bank:

Geography: France

Credit Score: 600

Gender: Male

Age: 40 years old

Tenure: 3 years

Balance: $ 60000

Number of Products: 2

Does this customer have a credit card? Yes

Is this customer an Active Member: Yes

Estimated Salary: $ 50000

So, should we say goodbye to that customer?

##### Predict method always takes a 2-D array hence the double square brackets
> We used feature scaling/sc object during training. Hence, we have to use that.

> We cannot use fit as it will cause leakage from the test data to the training data. Hence, we use transform.

> Also, France was respresented by (1,0,0) . Male was represented by (1)

In [27]:
print(ann.predict(sc.transform([[1,0,0,600,1,40,3,60000,2,1,1,50000]])))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
[[0.04428459]]


In [28]:
print(ann.predict(sc.transform([[1,0,0,600,1,40,3,60000,2,1,1,50000]]))>0.5)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
[[False]]


In [ ]:
#Therefore, we can say that the customer won't leave the bank, as the predicted probability is less than 0.5

In [29]:
y_pred= ann.predict(X_test)
y_pred=(y_pred>0.5)
#concatenating predicted and test values together
print(np.concatenate((y_pred.reshape(len(y_pred),1),y_test.reshape(len(y_test),1)),1))

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
[[0 0]
 [0 1]
 [0 0]
 ...
 [0 0]
 [0 0]
 [0 0]]


### Making the Confusion Matrix


In [31]:
from sklearn.metrics import confusion_matrix,accuracy_score
cm=confusion_matrix(y_test,y_pred)
print(cm)
accuracy_score(y_test,y_pred)

[[1505   90]
 [ 196  209]]


0.857

In [32]:
#Hence the final accuracy is 0.857